In [1]:
#pip install theano-pymc

In [2]:
#pip uninstall -y theano

In [3]:
#pip install pymc

In [19]:
import sys
import pandas as pd
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('functions')
from data_frame_handler import DataFrameHandler as dfh

# main path to file with data. Can be easily changed
file_path = 'data/urine_test_data.csv'

Step 0: Data prep

In [5]:
df = dfh.read_csv_data(file_path)

# Use a list comprehension to dynamically select matching columns
organisms = dfh.select_columns_by_search_terms(df, ['organism'])
genes = dfh.select_columns_by_search_terms(df, ['gene'])
antibiotics = dfh.select_columns_by_search_terms(df, ['antibiotic'])

# Set the 'Type' column to the values in gene_data_str - FUTURE Labels
gene_data_str = df[genes].applymap(str).agg(''.join, axis=1)
df['Type'] = gene_data_str

In [6]:
# Get a colormap based on the number of unique 'Type' values
geneset_unique_types = df['Type'].unique()
num_unique_types = len(geneset_unique_types)
cmap = plt.get_cmap('tab20', num_unique_types)

# Create a mapping of 'Type' values to colors
type_colors = {type_value: cmap(i) for i, type_value in enumerate(geneset_unique_types)}

# Map 'Type' values to colors in a new Series
type_colors_series = df['Type'].map(type_colors)

In [7]:
organisms = dfh.select_columns_by_search_terms(df, ['Organism_1', 'Organism_2', 'Organism_3'] )




In [ ]:
# Iterate through pairs of organisms and antibiotics
for org, anti in zip(organisms, antibiotics):

    # Extract data for the current pair
    X = df[org].values.astype(float)
    y = df[anti].replace({'R': 1, 'S': 0}).values
    
    # Bayesian Linear Regression Model
    with pm.Model() as model:
        
        # Priors
        alpha = pm.Normal('alpha', mu=0, sigma=10)
        beta = pm.Normal('beta', mu=0, sigma=10)
        sigma = pm.HalfNormal('sigma', sigma=1)
        
        # Likelihood
        mu = alpha + beta * X
        likelihood = pm.Normal('y', mu=mu, sigma=sigma, observed=y)
        
        # MCMC
        trace = pm.sample(2000, tune=1000)
        
        # Visualization
        # Trace plot
        pm.traceplot(trace)
        plt.title(f"Traceplot for {org} vs {anti}")
        plt.show()

        # Posterior plot
        pm.plot_posterior(trace)
        plt.title(f"Posterior distributions for {org} vs {anti}")
        plt.show()
        
        # Predictive check
        pm.plot_ppc(trace)
        plt.title(f"Predictive check for {org} vs {anti}")
        plt.show()


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, beta, sigma]


/usr/local/lib/python3.8/site-packages/pytensor/scalar/basic.py:3092: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/usr/local/lib/python3.8/site-packages/pytensor/scalar/basic.py:3092: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/usr/local/lib/python3.8/site-packages/pytensor/scalar/basic.py:3198: RuntimeWarning: overflow encountered in scalar multiply
  return x * x


In [14]:
# Iterate through pairs of organisms and antibiotics
for org, anti in zip(organisms, antibiotics):

    # Extract data for the current pair
    X = df[org].values
    y = df[anti].replace({'R': 1, 'S': 0}).values
    
    # Bayesian Linear Regression Model
    with pm.Model() as model:

        # Updated Priors
        alpha = pm.Uniform('alpha', lower=-10, upper=10)
        beta = pm.Exponential('beta', lam=1)
        sigma = pm.HalfCauchy('sigma', beta=2.5)
        
        # Likelihood
        mu = alpha + beta * X
        likelihood = pm.Normal('y', mu=mu, sd=sigma, observed=y)
        
        # MCMC
        trace = pm.sample(2000, tune=1000)

TypeError: make_node() got an unexpected keyword argument 'sd'